In [2]:
from getpass import getpass
from base64 import encode
from email import message
import secrets
from Crypto.Random import get_random_bytes
from Crypto.Protocol.KDF import PBKDF2
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad

In [31]:
def gen_key(path='key.bin',type_passwd=True, import_secret='magic'):
    simple_key = get_random_bytes(32)
    salt = simple_key
    if type_passwd == True:
        password = getpass()
    if type_passwd == False:
        password = import_secret
    key = PBKDF2(password, salt, dkLen=32)
    with open(path,'wb') as f:
        f.write(salt)
    print("YOUR KEY WAS CREATED BY MAGIC!")

In [33]:
def get_key(key_file='key.bin', type_password=True, import_secret='magic'):
    with open(key_file,'rb') as f:
        salt = f.read()
    if type_password == True:
        password = getpass()
    if type_password == False:
        password = import_secret
    key = PBKDF2(password, salt, dkLen=32)
    print("THE KEY IS IN YOUR HANDS!")
    return key

In [18]:
def encrypt_file(secret_file='input', key_file='key.bin',import_secret='magic', output_file='encrypted.bin'):
    with open(secret_file,'rb') as f:
        dark_secret =  f.read()
    
    key = get_key(key_file='key.bin', type_password=True, import_secret='magic')
    cipher = AES.new(key, AES.MODE_CBC)
    ciphered_data = cipher.encrypt(pad(dark_secret, AES.block_size))
   
    with open(output_file, 'wb') as f:
        f.write(cipher.iv)
        f.write(ciphered_data)

    print("YOUR SECRET IS SAFE IN THE SHADOWS!")

In [19]:
def decrypt_file(secret_file='encrypted.bin', key_file='key.bin',old_secret='magic'):
    print("THE BLACK BOX IS OPENED!\n")
    key = get_key(key_file='key.bin', type_secret=True, old_secret='magic')
    
    with open(secret_file, 'rb') as f:
        iv = f.read(16)
        decrypt_data = f.read()

    cipher = AES.new(key, AES.MODE_CBC, iv=iv)
    original = unpad(cipher.decrypt(decrypt_data), AES.block_size)
    return original.decode('utf-8')